In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
from itertools import groupby
import numpy as np
import cv2
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
from glob import glob
from multiprocessing import Pool
import matplotlib.pyplot as plt
# import cupy as cp
import ast
from pathlib import Path
import pickle5 as pickle
import torch

import shutil
import sys
sys.path.append("../src/")
sys.path.append("../yolov5/")

import util

from joblib import Parallel, delayed

from IPython.display import display, HTML
from sahi.model import Yolov5DetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from ensemble_boxes import weighted_boxes_fusion

import albumentations as A

from IPython.display import Image
from matplotlib import animation, rc
rc('animation', html='jshtml')

In [3]:
CONF = 0
IOU = 0.4
AUGMENT = True # True 
SIZE = int(1280 * 1.3)  # 1280 * 1.3 
#USE_CLACHE = True
USE_SAHI = False

# Load Data

In [4]:
df = util.load_pickle("pred_CV3_0128.pkl")
# df_2 = util.load_pickle("pred_CV3_s1800_0129.pkl")
# for col in df_2.columns:
#     if col not in df.columns:
#         df[col] = df_2[col]
df_3 = util.load_pickle("pred_CV3_s1800_e11_0129.pkl")
for col in df_3.columns:
    if col not in df.columns:
        df[col] = df_3[col]
df_4 = util.load_pickle("pred_CV3_s1280_clahe_0129.pkl")
for col in df_4.columns:
    if col not in df.columns:
        df[col] = df_4[col]
        
# df_5 = util.load_pickle("pred_CV3_s3600_e11_0129.pkl")
# df_5 = df_5.rename(columns = {"pred_0129_yolov5l_v3_highFP_1800_e11_fold1":"pred_0129_yolov5l_v3_highFP_3600_e11_fold1" })
# for col in df_5.columns:
#     if col not in df.columns:
#         df[col] = df_5[col]

df_6 = util.load_pickle("pred_CV3_s1800_clahe_0129.pkl")
for col in df_6.columns:
    if col not in df.columns:
        df[col] = df_6[col]
        
df_7 = util.load_pickle("pred_CV3_s2700_clahe_0129.pkl")
df_7 = df_7.rename(columns = {"pred_0129_yolov5l_v3_1800_clahe_fold1":"pred_0129_yolov5l_v3_2700_clahe_fold1" })
for col in df_7.columns:
    if col not in df.columns:
        df[col] = df_7[col]

In [5]:
model_versions = ["_".join(col.split("_")[1:-1]) for col in df.columns if "_fold1" in col]
model_versions

['0128_yolov5l_v3',
 '0128_yolov5l_v3_highFP',
 '0129_yolov5l_v3_highFP_1800_e11',
 '0129_yolov5l_v3_highFP_clahe',
 '0129_yolov5l_v3_1800_clahe',
 '0129_yolov5l_v3_2700_clahe']

# Model

# Pick Conf Thresh

In [6]:
df_oof = df.query("fold == 1").copy()
gt_bboxs_list = []
for index in df_oof.index:
    gt_bboxs_list.append(np.array(df_oof.loc[index, "real_bbox"]))

In [7]:
#conf_ths = [5, 10, 15, 20, 25, 30, 35, 40]
conf_ths = np.arange(15, 25, 1)
#conf_ths = [12, 16, 18, 20, 22, 24]
#conf_ths = [6, 8, 10, 12, 14]
#conf_ths = [30, 32, 35, 38, 45, 50, 55]
iou_ths = np.arange(0.3, 0.85, 0.05)
fold = 1
for model_version in model_versions:
    if not model_version in ['0129_yolov5l_v3_2700_clahe']:
        continue
    pred_col = "pred_" + model_version + f"_fold{fold}"
    for conf_th in conf_ths:
        conf_th /= 100
        prd_bboxs_list = []
        for index in df_oof.index:
            prd_bboxs = df_oof.loc[index, pred_col]
            prd_bboxs = [p for p in prd_bboxs if p[0] >= conf_th]
            prd_bboxs_list.append(np.array(prd_bboxs))
        f2_dict = util.calc_f2_score(gt_bboxs_list, prd_bboxs_list, verbose=False)    
        print("model {}, conf_th {:.2f}, f2 {:.3f}, precision {:.3f}, recall {:.3f}".format(model_version, conf_th, f2_dict['f2'], f2_dict['P'], f2_dict['R']))

model 0129_yolov5l_v3_2700_clahe, conf_th 0.15, f2 0.637, precision 0.618, recall 0.642
model 0129_yolov5l_v3_2700_clahe, conf_th 0.16, f2 0.637, precision 0.628, recall 0.639
model 0129_yolov5l_v3_2700_clahe, conf_th 0.17, f2 0.637, precision 0.641, recall 0.636
model 0129_yolov5l_v3_2700_clahe, conf_th 0.18, f2 0.638, precision 0.649, recall 0.635
model 0129_yolov5l_v3_2700_clahe, conf_th 0.19, f2 0.639, precision 0.660, recall 0.634
model 0129_yolov5l_v3_2700_clahe, conf_th 0.20, f2 0.638, precision 0.665, recall 0.631
model 0129_yolov5l_v3_2700_clahe, conf_th 0.21, f2 0.639, precision 0.674, recall 0.631
model 0129_yolov5l_v3_2700_clahe, conf_th 0.22, f2 0.638, precision 0.680, recall 0.628
model 0129_yolov5l_v3_2700_clahe, conf_th 0.23, f2 0.638, precision 0.688, recall 0.627
model 0129_yolov5l_v3_2700_clahe, conf_th 0.24, f2 0.638, precision 0.691, recall 0.626


In [8]:
best_conf_dict = {
    '0128_yolov5l_v3': 0.25,
    "0128_yolov5l_v3_highFP": 0.25,
    "0129_yolov5l_v3_highFP_1800": 0.25,
    "0129_yolov5l_v3_highFP_1800_e11": 0.16,
    "0129_yolov5l_v3_highFP_clahe": 0.19,
    '0129_yolov5l_v3_highFP_3600_e11': 0.25,
    '0129_yolov5l_v3_1800_clahe': 0.15,
    '0129_yolov5l_v3_2700_clahe': 0.2,
}

In [9]:
pd.set_option('display.max_columns', 40)
f_cols = ["f2"] + ["f2_" + str(round(th,3)) for th in np.arange(0.3, 0.85, 0.05)]
P_cols = ["P"] + ["P_" + str(round(th,3)) for th in np.arange(0.3, 0.85, 0.05)]
R_cols = ["R"] + ["R_" + str(round(th,3)) for th in np.arange(0.3, 0.85, 0.05)]

In [10]:
df_perf_dict = {model_version: pd.DataFrame(columns=["all"] + [str(round(th,3)) for th in np.arange(0.3, 0.85, 0.05)]).copy() for model_version in model_versions}
fold = 1
for model_version in model_versions:
    pred_col = "pred_" + model_version + f"_fold{fold}"
    conf_th = best_conf_dict[model_version]
    prd_bboxs_list = []
    for index in df_oof.index:
        prd_bboxs = df_oof.loc[index, pred_col]
        prd_bboxs = [p for p in prd_bboxs if p[0] >= conf_th]
        prd_bboxs_list.append(np.array(prd_bboxs))
    f2_dict = util.calc_f2_score(gt_bboxs_list, prd_bboxs_list, verbose=False)    
    df_perf_dict[model_version].loc["f2",:] = [f2_dict[col] for col in f_cols]
    df_perf_dict[model_version].loc["P",:] = [f2_dict[col] for col in P_cols]
    df_perf_dict[model_version].loc["R",:] = [f2_dict[col] for col in R_cols]
    print("model {}, conf_th {:.2f}, f2 {:.3f}, precision {:.3f}, recall {:.3f}".format(model_version, conf_th, f2_dict['f2'], f2_dict['P'], f2_dict['R']))
    display(df_perf_dict[model_version].astype(float).style.format("{:.3f}").background_gradient(axis=1))

model 0128_yolov5l_v3, conf_th 0.25, f2 0.649, precision 0.576, recall 0.670


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.649,0.744,0.744,0.744,0.744,0.741,0.731,0.716,0.668,0.579,0.434,0.294
P,0.576,0.660,0.660,0.660,0.660,0.657,0.648,0.635,0.592,0.514,0.385,0.261
R,0.670,0.769,0.769,0.769,0.768,0.765,0.755,0.739,0.689,0.598,0.448,0.304


model 0128_yolov5l_v3_highFP, conf_th 0.25, f2 0.642, precision 0.710, recall 0.628


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.642,0.737,0.737,0.737,0.735,0.727,0.720,0.696,0.646,0.572,0.450,0.311
P,0.710,0.814,0.814,0.814,0.811,0.803,0.795,0.769,0.714,0.632,0.497,0.343
R,0.628,0.720,0.720,0.720,0.718,0.710,0.703,0.680,0.631,0.558,0.440,0.304


model 0129_yolov5l_v3_highFP_1800_e11, conf_th 0.16, f2 0.645, precision 0.630, recall 0.649


...

,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.625,0.714,0.714,0.714,0.714,0.714,0.707,0.685,0.642,0.558,0.438,0.274
P,0.646,0.738,0.738,0.738,0.738,0.738,0.731,0.708,0.664,0.577,0.453,0.284
R,0.620,0.709,0.709,0.709,0.709,0.709,0.702,0.679,0.637,0.553,0.435,0.272


model 0129_yolov5l_v3_1800_clahe, conf_th 0.15, f2 0.643, precision 0.663, recall 0.638


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.643,0.758,0.758,0.758,0.757,0.751,0.740,0.706,0.639,0.540,0.415,0.253
P,0.663,0.782,0.782,0.782,0.780,0.774,0.764,0.728,0.659,0.556,0.428,0.261
R,0.638,0.752,0.752,0.752,0.751,0.745,0.735,0.700,0.635,0.535,0.412,0.251


model 0129_yolov5l_v3_2700_clahe, conf_th 0.20, f2 0.638, precision 0.665, recall 0.631


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.638,0.759,0.759,0.759,0.757,0.745,0.727,0.689,0.620,0.530,0.405,0.267
P,0.665,0.792,0.792,0.792,0.789,0.777,0.758,0.719,0.647,0.553,0.422,0.278
R,0.631,0.751,0.751,0.751,0.749,0.737,0.719,0.682,0.614,0.525,0.401,0.264


# Tracking

In [11]:
from norfair import Detection, Tracker

# Helper to convert bbox in format [x_min, y_min, x_max, y_max, score] to norfair.Detection class
def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))
        
    return result

def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)

def tracking_function(tracker, frame_id, bboxes, scores, best_conf):
    
    detects = []
    predictions = []
    
    if len(scores)>0:
        for i in range(len(bboxes)):
            # remember to check
            if scores[i] <= best_conf:
                continue
            box = bboxes[i]
            score = scores[i]
            x_min = int(box[0])
            y_min = int(box[1])
            bbox_width = int(box[2])
            bbox_height = int(box[3])
            detects.append([x_min, y_min, x_min+bbox_width, y_min+bbox_height, score])
            predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
#             print(predictions[:-1])
    # Update tracks using detects from current frame
    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id))
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
            continue
        # Add objects that have no detections on current frame to predictions
        xc, yc = tobj.estimate[0]
        x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))
        
        #exclude those in the edge
        if (x_min + bbox_width >= 1280) or (y_min + bbox_height) >= 720 or (x_min <= 0) or (y_min <= 0):
            continue
        score = tobj.last_detection.scores[0]

        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
        
    return predictions

In [12]:
df_perf_track_dict = {model_version: pd.DataFrame(columns=["all"] + [str(round(th,3)) for th in np.arange(0.3, 0.85, 0.05)]).copy() for model_version in model_versions}
df_oof.sort_values(["video_id","video_frame"], inplace=True)
df_oof["track_pred"] = [[] for i in range(len(df_oof))]
for model_version in model_versions:
    BEST_CONF = best_conf_dict[model_version]
    pred_col = "pred_" + model_version + f"_fold1"
    previous_seq = -1
    prd_bboxs_tracking_list = []
    frame_id = 0
    tracker = Tracker(
        distance_function=euclidean_distance, 
        distance_threshold=30,
        hit_inertia_min=3,
        hit_inertia_max=6,
        initialization_delay=1,
    )          
    for idx in df_oof.index:
        row = df_oof.loc[idx]
        if row["sequence"] != previous_seq:
            previous_seq = row["sequence"]
        prd_bboxs = row[pred_col]
        bboxes, confis = [p[1:] for p in prd_bboxs], [p[0] for p in prd_bboxs]    
        predictions = tracking_function(tracker, frame_id, bboxes, confis, BEST_CONF)    
        prd_bboxs_tracking =  [[float(p) for p in pred.split(" ")] for pred in predictions]
        prd_bboxs_tracking_list.append(np.array(prd_bboxs_tracking))
        df_oof.at[idx, "track_pred"].extend(prd_bboxs_tracking)
        frame_id += 1        
        
    f2_dict = util.calc_f2_score(gt_bboxs_list, prd_bboxs_tracking_list, verbose=False) 
    df_perf_track_dict[model_version].loc["f2",:] = [f2_dict[col] for col in f_cols]
    df_perf_track_dict[model_version].loc["P",:] = [f2_dict[col] for col in P_cols]
    df_perf_track_dict[model_version].loc["R",:] = [f2_dict[col] for col in R_cols]
    df_oof[f'{model_version}_tps'] = f2_dict["tps"]
    df_oof[f'{model_version}_fps'] = f2_dict["fps"]
    df_oof[f'{model_version}_fns'] = f2_dict["fns"]
    print("Using Tracking new: model {}, conf_th {:.2f}, f2 {:.3f}, precision {:.3f}, recall {:.3f}".format(model_version, BEST_CONF, f2_dict['f2'], f2_dict['P'], f2_dict['R']))
    display(df_perf_track_dict[model_version].astype(float).style.format("{:.3f}").background_gradient(axis=1))

Using Tracking new: model 0128_yolov5l_v3, conf_th 0.25, f2 0.653, precision 0.519, recall 0.698


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.653,0.762,0.760,0.759,0.757,0.751,0.734,0.716,0.664,0.570,0.425,0.286
P,0.519,0.606,0.604,0.603,0.602,0.597,0.584,0.569,0.528,0.453,0.338,0.228
R,0.698,0.815,0.812,0.811,0.809,0.803,0.785,0.765,0.710,0.610,0.454,0.306


Using Tracking new: model 0128_yolov5l_v3_highFP, conf_th 0.25, f2 0.655, precision 0.660, recall 0.653


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.655,0.762,0.761,0.759,0.753,0.743,0.731,0.707,0.654,0.574,0.448,0.309
P,0.660,0.769,0.767,0.766,0.760,0.750,0.738,0.713,0.660,0.579,0.452,0.311
R,0.653,0.760,0.759,0.757,0.751,0.742,0.730,0.705,0.653,0.573,0.447,0.308


Using Tracking new: model 0129_yolov5l_v3_highFP_1800_e11, conf_th 0.16, f2 0.659, precision 0.588, recall 0.679


...

,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.631,0.735,0.734,0.729,0.727,0.723,0.711,0.686,0.640,0.554,0.434,0.271
P,0.598,0.697,0.696,0.691,0.689,0.685,0.674,0.651,0.607,0.525,0.411,0.257
R,0.640,0.745,0.744,0.739,0.737,0.733,0.721,0.696,0.649,0.562,0.440,0.275


Using Tracking new: model 0129_yolov5l_v3_1800_clahe, conf_th 0.15, f2 0.649, precision 0.617, recall 0.657


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.649,0.779,0.777,0.773,0.768,0.759,0.742,0.706,0.637,0.536,0.411,0.249
P,0.617,0.741,0.739,0.736,0.730,0.722,0.706,0.672,0.606,0.510,0.391,0.237
R,0.657,0.789,0.787,0.783,0.778,0.769,0.752,0.715,0.645,0.543,0.417,0.252


Using Tracking new: model 0129_yolov5l_v3_2700_clahe, conf_th 0.20, f2 0.646, precision 0.616, recall 0.654


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.646,0.785,0.783,0.780,0.773,0.759,0.733,0.689,0.617,0.526,0.398,0.261
P,0.616,0.749,0.747,0.744,0.738,0.724,0.700,0.657,0.589,0.502,0.380,0.249
R,0.654,0.795,0.792,0.790,0.783,0.769,0.742,0.698,0.625,0.532,0.403,0.265


# Ensemble

In [ ]:
from ensemble_boxes import weighted_boxes_fusion
def run_wbf(bboxes, confs, iou_thr=0.5, skip_box_thr=0.00000001):
    if len(bboxes) == 1:
        return bboxes, confs, []    
    labels = [np.ones(len(conf)) for conf in confs]
    boxes, scores, labels = weighted_boxes_fusion(bboxes, confs, labels, iou_thr=iou_thr, skip_box_thr=skip_box_thr, allows_overflow=True)
    return boxes, scores, labels

In [ ]:
df_oof["wbf"] = None
wbf_models = ['0128_yolov5l_v3', '0128_yolov5l_v3_highFP', "0129_yolov5l_v3_highFP_1800_e11", '0129_yolov5l_v3_1800_clahe']
for iou_thr in [0.4]:
    iou_thr = iou_thr / 10
    for skip_box_thr in [0.2]:
        skip_box_thr = skip_box_thr / 10
        prd_wbf_list = []
        for idx in df_oof.index:
            row = df_oof.loc[idx]
            bboxes_l = []
            confs_l = []    
            frame_id = 0
            tracker = Tracker(
                distance_function=euclidean_distance, 
                distance_threshold=30,
                hit_inertia_min=3,
                hit_inertia_max=6,
                initialization_delay=1,
            )                      
            for model_version in wbf_models:
                BEST_CONF = best_conf_dict[model_version]
                pred_col = "pred_" + model_version + f"_fold1"
                prd_bboxs = row[pred_col]
                prd_bboxs = [p for p in prd_bboxs if p[0] >= BEST_CONF]
                bboxes, confis = [p[1:] for p in prd_bboxs], [p[0] for p in prd_bboxs]    
                bboxes = np.array(bboxes,dtype=np.float64)
                if len(bboxes) > 0:
                    bboxes[:,2:] = bboxes[:,2:] + bboxes[:,:2]
                    bboxes[:,0] = bboxes[:,0] / 1279.
                    bboxes[:,1] = bboxes[:,1] / 719.
                    bboxes[:,2] = bboxes[:,2] / 1279.
                    bboxes[:,3] = bboxes[:,3] / 719.
                bboxes_l.append(np.array(bboxes))
                confs_l.append(confis)
            bboxes_l = np.array(bboxes_l)
            wbf_boxes, wbf_confs, _ = run_wbf(bboxes_l, confs_l, iou_thr, skip_box_thr)
            if len(wbf_boxes) > 0:
                wbf_boxes[:,0] = wbf_boxes[:,0] * 1279.
                wbf_boxes[:,1] = wbf_boxes[:,1] * 719.
                wbf_boxes[:,2] = wbf_boxes[:,2] * 1279.
                wbf_boxes[:,3] = wbf_boxes[:,3] * 719.
                wbf_boxes[:,2:] = wbf_boxes[:,2:] - wbf_boxes[:,:2]
                
            predictions = tracking_function(tracker, frame_id, wbf_boxes, wbf_confs, 0)
            
            prd_bboxs_tracking =  [[float(p) for p in pred.split(" ")] for pred in predictions]
            prd_wbf_list.append(np.array(prd_bboxs_tracking))
            
            frame_id += 1

#             wbf_preds = [[wbf_confs[i]] + wbf_boxes[i].tolist() for i in range(len(wbf_boxes))]
#             prd_wbf_list.append(np.array(wbf_preds))
#             df_oof.at[idx, "wbf"] = wbf_preds

        f2_dict = util.calc_f2_score(gt_bboxs_list, prd_wbf_list, verbose=False) 
        print("iou_thr {:.2f}, skip_box_thr {:.2f}, Using wbf, f2 {:.3f}, precision {:.3f}, recall {:.3f}".format(iou_thr, skip_box_thr,f2_dict['f2'], f2_dict['P'], f2_dict['R']))

# check image

In [ ]:
df_oof['tp'] = 0
df_oof['fp'] = 0
df_oof['fn'] = 0
for key in model_versions:
    df_oof[f'{key}_tp'] = df_oof[f'{key}_tps'].apply(np.sum)
    df_oof[f'{key}_fp'] = df_oof[f'{key}_fps'].apply(np.sum)
    df_oof[f'{key}_fn'] = df_oof[f'{key}_fns'].apply(np.sum)
    df_oof['tp'] = df_oof['tp'] + df_oof[f'{key}_tp']
    df_oof['fp'] = df_oof['fp'] + df_oof[f'{key}_fp']
    df_oof['fn'] = df_oof['fn'] + df_oof[f'{key}_fn']

In [ ]:
color_pred = [(0,125,0)]    
color_real = [(255,0,0)]
def plot_img(df, idx, keys, save=False, pre=""):
    row = df.iloc[idx]
    path = row["image_path"]
    img = cv2.imread(str(path))[...,::-1]
    real_bboxes = row["real_bbox"]
    preds = row["track_pred"]
#     for key in keys:
#         best_conf = best_conf_dict[key]
#         pred_col = "pred_" + key + "_fold1"
#         if len(row[pred_col]) > 0:
#             preds = preds + [p for p in row[pred_col] if p[0] >= best_conf]
            
    bboxes = [p[1:5] for p in preds]
    confis = [p[0] for p in preds]
    print(f"video {row.video_id}, sequence {row.sequence}, frame {row.video_frame}")
    img_plot = util.show_img(img, 
                          bboxes + real_bboxes, 
                          confis + [1] * len(real_bboxes),
                          color_pred  * len(bboxes) + color_real * len(real_bboxes), 
                          bbox_format='coco')
    if save:
        img_plot.save(f"./pic_check/{pre}v{row.video_id}_frame{row.video_frame}_seq{row.sequence}.jpg")
    display(img_plot)

In [ ]:
for i, iou in tqdm(enumerate(np.arange(0.3,0.85,0.05))):
    df_oof[f"f2_{iou:.2f}"] = 0
    df_oof[f"P_{iou:.2f}"] = 0
    df_oof[f"R_{iou:.2f}"] = 0
    for idx in df_oof.index:
        tp = 0
        fp = 0
        fn = 0
        for key in model_versions:
            tp += df_oof.loc[idx, f'{key}_tps'][i]
            fp += df_oof.loc[idx, f'{key}_fps'][i]
            fn += df_oof.loc[idx, f'{key}_fns'][i]
        df_oof.loc[idx, f"f2_{iou:.2f}"] = util.f_beta(tp, fp, fn, beta=2)
        df_oof.loc[idx, f"P_{iou:.2f}"] = util.f_beta(tp, fp, fn, beta=0)
        df_oof.loc[idx, f"R_{iou:.2f}"] = util.f_beta(tp, fp, fn, beta=100)

In [ ]:
df_oof["f2"] = df_oof[[f'f2_{iou:.2f}' for iou in np.arange(0.3,0.85,0.5)]].mean(axis=1)
df_oof["P"] = df_oof[[f'P_{iou:.2f}' for iou in np.arange(0.3,0.85,0.5)]].mean(axis=1)
df_oof["R"] = df_oof[[f'R_{iou:.2f}' for iou in np.arange(0.3,0.85,0.5)]].mean(axis=1)
#df_oof.head()

In [ ]:
show_col = ["video_id","sequence","video_frame","num_bbox", "f2", "P", "R"]
sub_col = ["fp"] + [key+"_fp" for key in model_versions]
sub_col = sub_col + ["tp"] + [key+"_tp" for key in model_versions]
sub_col = sub_col + ["fn"] + [key+"_fn" for key in model_versions]

## most FP among empy group

In [ ]:
df_neg = df_oof.query("num_bbox == 0").copy()
df_neg.sort_values("fp", ascending=False, inplace=True)
df_neg.reset_index(inplace=True, drop=True)
df_neg[show_col + sub_col].head()

In [ ]:
for i in range(5):
    plot_img(df_neg, i, model_versions, save=False)

## Most FP among non-empty group

In [ ]:
df_pos = df_oof.query("num_bbox > 0").copy()
df_pos.sort_values("fp", ascending=False, inplace=True)
df_pos.reset_index(inplace=True, drop=True)
df_pos[show_col + sub_col].head()

In [ ]:
for i in range(5):
    plot_img(df_pos, i, model_versions)

## Most TP

In [ ]:
df_pos = df_oof.query("num_bbox > 0").copy()
df_pos.sort_values("tp", ascending=False, inplace=True)
df_pos.reset_index(inplace=True, drop=True)
df_pos[show_col + sub_col].head(5)

In [ ]:
for i in range(3):
    plot_img(df_pos, i, model_versions)

## Most FN

In [ ]:
df_pos = df_oof.query("num_bbox > 0").copy()
df_pos.sort_values("fn", ascending=False, inplace=True)
df_pos.reset_index(inplace=True, drop=True)
df_pos[show_col + sub_col].head(10)

In [ ]:
for i in range(10):
    plot_img(df_pos, i, model_versions)

# Improve FN

In [ ]:
egs = [
    # sequence , frame id
    (53708, 8926),
    (53708, 8928),
    (53708, 9654),
    (53708, 9655),
]

In [ ]:

def plot_img_2(df, sequence, frame_id, keys, save=False, pre="", use_clahe=False, aug=None):
    #row = df.iloc[idx]
    row = df.query("sequence == @seq and video_frame == @frame_id").iloc[0]
    path = row["image_path"]
    img = cv2.imread(str(path))[...,::-1]
    if use_clahe:
        for i in range(3):
            img[:, :, i] = clahe.apply((img[:, :, i]))         
    if aug is not None:
        img = aug(image=img)['image']
    real_bboxes = row["real_bbox"]
    preds = row["track_pred"]            
    bboxes = [p[1:5] for p in preds]
    confis = [p[0] for p in preds]
    print(f"video {row.video_id}, sequence {row.sequence}, frame {row.video_frame}")
    img_plot = util.show_img(img, 
                          bboxes + real_bboxes, 
                          confis + [1] * len(real_bboxes),
                          color_pred  * len(bboxes) + color_real * len(real_bboxes), 
                          bbox_format='coco')
    if save:
        img_plot.save(f"./pic_check/{pre}v{row.video_id}_frame{row.video_frame}_seq{row.sequence}.jpg")
    display(img_plot)

In [ ]:
seq, frame = egs[0]
seq_tp = 53708
frame_tp = 9653
plot_img_2(df_oof, seq, frame, model_versions, use_clahe=False)
plot_img_2(df_oof, seq_tp, frame_tp, model_versions, use_clahe=False)

## CLAHE

In [ ]:
clahe = cv2.createCLAHE(clipLimit=5, tileGridSize=(10, 10))
plot_img_2(df_oof, seq, frame, model_versions, use_clahe=True)
plot_img_2(df_oof, seq_tp, frame_tp, model_versions, use_clahe=True)        

## HueSaturationValue

In [ ]:
aug = A.HueSaturationValue(hue_shift_limit=10,p = 1.0)
plot_img_2(df_oof, seq, frame, model_versions, use_clahe=False, aug=aug)
plot_img_2(df_oof, seq_tp, frame_tp, model_versions, use_clahe=False, aug=aug)        

## RGBShift

In [ ]:
aug = A.RGBShift(r_shift_limit = 40, b_shift_limit=0, g_shift_limit=0, p = 1.0)
plot_img_2(df_oof, seq, frame, model_versions, use_clahe=False, aug=aug)
plot_img_2(df_oof, seq_tp, frame_tp, model_versions, use_clahe=False, aug=aug)        

## FUnIE-GAN

In [ ]:
sys.path.append("./FUnIE-GAN/PyTorch/")
from nets import funiegan

model_path = "./FUnIE-GAN/PyTorch/models/funie_generator.pth"
model = funiegan.GeneratorFunieGAN()
model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))

In [ ]:
model.eval()

In [ ]:
import torchvision.transforms as transforms
## data pipeline
img_width, img_height, channels = 1280, 1280, 3
transforms_ = [transforms.Resize((img_height, img_width), transforms.InterpolationMode.BICUBIC),
               transforms.ToTensor(),
               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]
transform = transforms.Compose(transforms_)

In [ ]:
path = df_oof.query("sequence == 53708 and video_frame == 8926").iloc[0]["image_path"]
path = df_oof.query("sequence == 53708 and video_frame == 9653").iloc[0]["image_path"]

In [ ]:
from PIL import Image
from torch.autograd import Variable
import torch
Tensor = torch.FloatTensor 
#path = '../../data/tensorflow-great-barrier-reef/train_images/video_0/8926.jpg'
inp_img = transform(Image.open(path))
inp_img = Variable(inp_img).type(Tensor).unsqueeze(0)
gen_img = model(inp_img)

In [ ]:
from torchvision.utils import save_image

In [ ]:
save_image(gen_img, "./test2.jpg", normalize=True)

In [ ]:
plt.imshow(gen_img.squeeze().data.permute(1, 2, 0).numpy())